In [1]:
import os.path, sys

import numpy as np
import pandas as pd
from scipy import linalg
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder

import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

from joblib import Parallel, delayed
import timeit

import matplotlib.pyplot as plt

# # --- Import our Code ---# #
#import emachine as EM
from direct_info import direct_info

# import data processing and general DCA_ER tools
from data_processing import data_processing_msa2pdb
import ecc_tools as tools
from pathlib import Path
np.random.seed(1)

from Bio import SeqIO
from Bio.PDB import *
from scipy.spatial import distance_matrix
from Bio import pairwise2
#from Bio.SubsMat.MatrixInfo import blosum62
pdb_parser = Bio.PDB.PDBParser()

from prody import *



In [2]:
# # # load and convert pfam to numpy array.
# msa_file = '/home/ecresswell/Pfam-A.full/PF00118_full.txt'

# from ecc_tools import read_FASTA
# msa_fasta, msa = read_FASTA(msa_file)
# np.save('/home/ecresswell/Pfam-A.full/PF00118/msa.npy', msa)


In [3]:
create_new = True
printing = True
removing_cols = True


data_path = Path('/home/ecresswell/Pfam-A.full')
data_path = Path('/data/cresswellclayec/Pfam-A.full')

# Define data directories
DCA_ER_dir = '/home/ecresswell/DCA_ER' # Set DCA_ER directory
DCA_ER_dir = '/data/cresswellclayec/DCA_ER' # Set DCA_ER directory

biowulf_dir = '%s/biowulf_full' % DCA_ER_dir

out_dir = '%s/protein_data/di/' % biowulf_dir
processed_data_dir = "%s/protein_data/data_processing_output" % biowulf_dir
pdb_dir = '%s/protein_data/pdb_data/' % biowulf_dir

pdb_path = "/pdb/pdb/zd/pdb1zdr.ent.gz"
# pdb_path = "/pdb/pdb/zd/pdb1zdr.ent.gz"
pdb_path_1 = "/pdb/pdb/ou/pdb2oug.ent.gz"
pdb_path_2 = "/pdb/pdb/ls/pdb2lcl.ent.gz"


pfam_dir = "/fdb/fastadb/pfam"

from data_processing import pdb2msa, data_processing_pdb2msa


import gzip, shutil
def gunzip(file_path, output_path):
    print('Unzipping %s to %s' % (file_path, output_path))
    with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
        shutil.copyfileobj(f_in, f_out)




n_cpus = 6
print('\n\nUnzipping %s' % pdb_path)





Unzipping /pdb/pdb/zd/pdb1zdr.ent.gz


In [4]:

unzipped_pdb_filename = os.path.basename(pdb_path_1).replace(".gz", "")

pdb_out_path = "%s%s" % (pdb_dir, unzipped_pdb_filename)
print('Unzipping %s to %s' % (pdb_path_1, pdb_out_path))

gunzip(pdb_path_1, pdb_out_path)
print(pdb_out_path)
print(pdb_dir)
pdb2msa_results_1 = pdb2msa(pdb_out_path, pdb_dir, create_new=False)
print(pdb2msa_results_1)


if len(pdb2msa_results_1) > 1:
    fasta_file = pdb2msa_results_1[0]
    prody_df = pdb2msa_results_1[1]
else:
    prody_df_1 = pdb2msa_results_1[0]


print('\nPDB DF with associated Protein Families\n', prody_df_1.loc[:,  [column for column in prody_df_1.columns if column not in ['locations', 'PDB Sequence']]].head())
print("\n\nLooping through Prody Search DataFrame:", prody_df_1.head())
rows_to_drop = []
for ir, pdb2msa_row in enumerate(prody_df_1.iterrows()):
    print('\n\nGetting msa with following pdb2msa entry:\n', pdb2msa_row)
    #try:
    dp_result1 =  data_processing_pdb2msa(data_path, prody_df_1.iloc[pdb2msa_row[0]], gap_seqs=0.2, gap_cols=0.2, prob_low=0.004,
                               conserved_cols=0.8, printing=True, out_dir=processed_data_dir, pdb_dir=pdb_dir, letter_format=False,
                               remove_cols=True, create_new=True, n_cpu=min(2, n_cpus))
    if dp_result1 is not None:
        [s0, removed_cols, s_index, tpdb, pdb_s_index] = dp_result1
        break
    else:
        rows_to_drop.append(ir)
        continue
    #except Exception as e:
    #    print('row %d got exception: ' % ir , e)
    #    print('moving on.. ')
    #    pass


pdb_id_1 = pdb2msa_row[1]['PDB ID']
pfam_id_1 = pdb2msa_row[1]['Pfam']
# update Prody search DF (use same filename as pdb2msa() in data_processing
prody_df_1 = prody_df_1.drop(rows_to_drop)

s0_1 = np.load("%s/%s_%s_preproc_msa.npy" % (processed_data_dir, pfam_id_1, pdb_id_1))
s_index_1 = np.load("%s/%s_%s_preproc_sindex.npy" % (processed_data_dir, pfam_id_1, pdb_id_1))
removed_cols_1 = np.load("%s/%s_%s_removed_cols.npy" % (processed_data_dir, pfam_id_1, pdb_id_1))
ref_seq_1 = np.load("%s/%s_%s_preproc_refseq.npy" % (processed_data_dir, pfam_id_1, pdb_id_1))



if dp_result1 is None:
    print('None of the available prody pdb search found matching alignments... Exiting..')
    sys.exit()
print('Done Preprocessing Data.....')


Unzipping /pdb/pdb/ou/pdb2oug.ent.gz to /data/cresswellclayec/DCA_ER/biowulf_full/protein_data/pdb_data/pdb2oug.ent
Unzipping /pdb/pdb/ou/pdb2oug.ent.gz to /data/cresswellclayec/DCA_ER/biowulf_full/protein_data/pdb_data/pdb2oug.ent
/data/cresswellclayec/DCA_ER/biowulf_full/protein_data/pdb_data/pdb2oug.ent
/data/cresswellclayec/DCA_ER/biowulf_full/protein_data/pdb_data/
2oug
Record id 2OUG:A, chain A
['UNP:P0AFW0', 'UNP:RFAH_ECOLI']
Record id 2OUG:B, chain B
['UNP:P0AFW0', 'UNP:RFAH_ECOLI']
Record id 2OUG:C, chain C
['UNP:P0AFW0', 'UNP:RFAH_ECOLI']
Record id 2OUG:D, chain D
['UNP:P0AFW0', 'UNP:RFAH_ECOLI']

Chain A polypeptide 0 (length 99):  QSWYLLYCKRGQLQRAQEHLERQAVNCLAPMITLEKIVRGKRTAVSEPLFPNYLFVEFDPEVIHTTTINATRGVSHFVRFGASPAIVPSAVIHQLSVYK


@> Submitted Pfam search for sequence "QSWYLLYCKRGQLQRA...".


{'PF02357': {'accession': 'PF02357.22', 'class': 'Domain', 'id': 'NusG', 'locations': {'ali_end': '94', 'ali_start': '2', 'bitscore': '69.18', 'end': '96', 'cond_evalue': '1.9e-23', 'ind_evalue': '3.7e-19', 'evidence': 'hmmer v3.0', 'hmm_end': '96', 'hmm_start': '2', 'start': '1'}, 'type': 'Pfam-A'}}

Chain A polypeptide 1 (length 42):  KVIITEGAFEGFQAIFTEPDGEARSMLLLNLINKEIKHSVKN


@> Submitted Pfam search for sequence "KVIITEGAFEGFQAIF...".


ValueError: No matching Pfam domains were found.

In [ ]:
unzipped_pdb_filename = os.path.basename(pdb_path_2).replace(".gz", "")

pdb_out_path = "%s%s" % (pdb_dir, unzipped_pdb_filename)
print('Unzipping %s to %s' % (pdb_path_2, pdb_out_path))

gunzip(pdb_path_2, pdb_out_path)
print(pdb_out_path)
print(pdb_dir)
pdb2msa_results_2 = pdb2msa(pdb_out_path, pdb_dir, create_new=False)
print(pdb2msa_results_2)


if len(pdb2msa_results_2) > 1:
    fasta_file = pdb2msa_results_2[0]
    prody_df = pdb2msa_results_2[1]
else:
    prody_df_2 = pdb2msa_results_2[0]


print('\nPDB DF with associated Protein Families\n', prody_df_2.loc[:,  [column for column in prody_df_2.columns if column not in ['locations', 'PDB Sequence']]].head())
print("\n\nLooping through Prody Search DataFrame:", prody_df_2.head())
rows_to_drop = []
for ir, pdb2msa_row in enumerate(prody_df_2.iterrows()):
    print('\n\nGetting msa with following pdb2msa entry:\n', pdb2msa_row)
    #try:
    dp_result2 =  data_processing_pdb2msa(data_path, prody_df_1.iloc[pdb2msa_row[0]], gap_seqs=0.2, gap_cols=0.2, prob_low=0.004,
                               conserved_cols=0.8, printing=True, out_dir=processed_data_dir, pdb_dir=pdb_dir, letter_format=False,
                               remove_cols=True, create_new=True, n_cpu=min(2, n_cpus))
    if dp_result2 is not None:
        [s0, removed_cols, s_index, tpdb, pdb_s_index] = dp_result2
        break
    else:
        rows_to_drop.append(ir)
        continue
    #except Exception as e:
    #    print('row %d got exception: ' % ir , e)
    #    print('moving on.. ')
    #    pass


pdb_id_2 = pdb2msa_row[1]['PDB ID']
pfam_id_2 = pdb2msa_row[1]['Pfam']
# update Prody search DF (use same filename as pdb2msa() in data_processing
prody_df_2 = prody_df_2.drop(rows_to_drop)

s0_2 = np.load("%s/%s_%s_preproc_msa.npy" % (processed_data_dir, pfam_id_2, pdb_id_2))
s_index_2 = np.load("%s/%s_%s_preproc_sindex.npy" % (processed_data_dir, pfam_id_2, pdb_id_2))
removed_cols_2 = np.load("%s/%s_%s_removed_cols.npy" % (processed_data_dir, pfam_id_2, pdb_id_2))
ref_seq_2 = np.load("%s/%s_%s_preproc_refseq.npy" % (processed_data_dir, pfam_id_2, pdb_id_2))


if dp_result2 is None:
    print('None of the available prody pdb search found matching alignments... Exiting..')
    sys.exit()
print('Done Preprocessing Data.....')

In [ ]:
# Load ER results for both pfam-pdb pairs 
# should all be the same
print('%s %s\n%s %s' % (pdb_id_1, pfam_id_1, pdb_id_2, pfam_id_2))
ER_di_1 = np.load("%s/%s_%s_ER_di.npy" % (out_dir, pdb_id_1, pfam_id_1))
ER_di_2 = np.load("%s/%s_%s_ER_di.npy" % (out_dir, pdb_id_2, pfam_id_2))
print(dp_result1[2], len(dp_result1[2]))
print(dp_result2[2], len(dp_result2[2]))
print(dp_result1[0][dp_result1[3]])
print(dp_result2[0][dp_result2[3]])

### plot contact maps and auc curves to see which structure ER is capturing better